In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df=pd.read_csv("../input/lazada-indonesian-reviews/20191002-reviews.csv")
data=df.copy()
data=data[['reviewContent','rating']]
data=data.dropna()
data.head()

In [ ]:
data['rating'].value_counts().sort_values(ascending=True)

In [ ]:
plot=data.rating.value_counts()
px.bar(plot,color=plot.index,color_continuous_scale=px.colors.sequential.Viridis)

In [ ]:
label=[]
for index, row in data.iterrows():
    if row['rating']==5:
        label.append(1)
    else:
        label.append(0)

data['label']=label
data=data.drop(columns='rating', axis=1)
data.head()

In [ ]:
plot=data.label.value_counts()
px.bar(plot,color=plot.index,color_continuous_scale=px.colors.sequential.Cividis)

In [ ]:
data

In [ ]:
s1=data[data.label==1].sample(25000, replace=True)
s2=data[data.label==0].sample(25000, replace=True)

data=pd.concat([s1,s2])
print(data.shape)
print(data.label.value_counts(normalize=True))

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
#polarity = 0
trains1=data[data['label']==0]
all_text=' '.join(word for word in trains1['reviewContent'])
word_cloud=WordCloud(colormap='Reds', width=1000, height=1000,mode='RGBA',background_color='white').generate(all_text)
plt.figure(figsize=(20,20))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
plt.show()

In [ ]:
#polarity = 1
trains1=data[data['label']==1]
all_text=' '.join(word for word in trains1['reviewContent'])
word_cloud=WordCloud(colormap='Blues', width=1000, height=1000,mode='RGBA',background_color='white').generate(all_text)
plt.figure(figsize=(20,20))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
plt.show()

In [ ]:
import string, re

def cleansing(data):
    #lowertext
    data=data.lower()
    
    #Remove Punctuation
    remove=string.punctuation
    translator=str.maketrans(remove,' '*len(remove))
    data=data.translate(translator)
    
    #Remove ASCII & UNICODE
    data=data.encode('ascii','ignore').decode('utf-8')
    data=re.sub(r'[^\x00-\x7f]',r'', data)
    
    #Remove Newline
    data=data.replace('\n',' ')
    
    return data




In [ ]:
review = []
for index, row  in data.iterrows():
    review.append(cleansing(row['reviewContent']))


In [ ]:
review

In [ ]:
pip install Sastrawi

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
 
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
 
# Contoh
sentence = 'alhamdulillah  sampai dengan selamat   smoga awet   terima kasih lazada retail coocaa '
stop = stopword.remove(sentence)
print(stop)

In [ ]:
review = []
for index, row in data.iterrows():
    review.append(stopword.remove(row['reviewContent']))


data['reviewContent']=review
data

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# contoh
sentence = 'packing rapi menggunakan kayu  mantap   dapet headphone'
s_clean = stemmer.stem(sentence)
 
print(s_clean)

In [ ]:
#Stemming
from tqdm import tqdm
review = []
for index, row in tqdm(data.iterrows()):
    review.append(stemmer.stem(row["reviewContent"]))
    
data['reviewContent']=review
data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['reviewContent'], data['label'], 
                                                    test_size=0.1, stratify=data['label'], random_state=30)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'Aku makan nasi',
    'Andi makan nasi uduk',
    'Nasi uduk sangat enak',
    'Enak banget nasi goreng']

vectorizer = TfidfVectorizer()

# contoh
X = vectorizer.fit_transform(corpus)
X.toarray()

In [ ]:
# implementasi pada dokumen kita
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

clf = svm.SVC(kernel="linear")

#cross_val_score(clf, X_train, y_train, cv=10)

In [ ]:
#Fitting
clf.fit(X_train,y_train)
predict = clf.predict(X_test)

In [ ]:
# import library evaluation
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, accuracy_score

In [ ]:
# f1_score
print("f1 score hasil prediksi adalah: ")
print(f1_score(y_test, predict))

# accuracy score
print("accuracy score hasil prediksi adalah: ")
print(accuracy_score(y_test, predict))

# precision score
print("precision score hasil prediksi adalah: ")
print(precision_score(y_test, predict))

# recall score
print("recall score hasil prediksi adalah: ")
print(recall_score(y_test, predict))

In [ ]:
import seaborn as sns
_=sns.heatmap(confusion_matrix(y_test, predict), annot=True,fmt='g')